In [25]:
%matplotlib inline
import os
import csv
import numpy as np
import re
import datetime
import fnmatch
from pathlib import Path
from itertools import groupby
import pandas as p
import xlrd

import distance
import nltk
#nltk.download('stopwords')

from nltk.stem import SnowballStemmer
from nltk.stem import snowball

In [2]:
def FixScratchPad(ScratchPad_Old):
    # Fix the situation where comma has divided decimals into separate columns
    
    ScratchPad_Times = [item[0] for item in ScratchPad_Old]
    
    ScratchPad_Phrases = list()
    
    # loop to combine phrases divided by commas
    ScratchPadInd = -1 
    while ScratchPadInd < len(ScratchPad_Old)-1:
        ScratchPadInd = ScratchPadInd + 1
        commasInPhrase = len(ScratchPad_Old[ScratchPadInd])-2
        if commasInPhrase < 1:
            #print(ScratchPad_Old[ScratchPadInd][1])
            ScratchPad_Phrases.append(ScratchPad_Old[ScratchPadInd][1])
            continue
        scratchPadPhrase = ScratchPad_Old[ScratchPadInd][1]
        for phraseJoinNr in range(1, commasInPhrase+1):
            scratchPadPhrase = scratchPadPhrase + ', ' + ScratchPad_Old[ScratchPadInd][1+phraseJoinNr]
        
        ScratchPad_Phrases.append(scratchPadPhrase)
            
        
    ScratchPad_New = [[ScratchPad_Times[ind], ScratchPad_Phrases[ind]] for ind in 
                    range(0, len(ScratchPad_Times))]
    
    #UserKeys_New = np.concatenate((UserKeys_Times, UserKeys_Keys, UserKeys_ProgressPercent), axis = 0)
    
    #print(ScratchPad_New)
    return ScratchPad_New

In [3]:
def FixUserKeys(UserKeys_Old):
    # Fix the situation where comma has divided decimals into separate columns
    
    
    Column_beforeDecimal = [item[2] for item in UserKeys_Old]
    Column_afterDecimal = [item[3] if len(item)>3 else '00' for item in UserKeys_Old]
    
    UserKeys_ProgressPercent = [float(Column_beforeDecimal[i]+'.'+ Column_afterDecimal[i]) for i in 
                                range(0, len(Column_beforeDecimal))]
    UserKeys_Times = [item[0] for item in UserKeys_Old]
    UserKeys_Keys = [item[1] for item in UserKeys_Old]
    
    UserKeys_New = [[UserKeys_Times[ind], UserKeys_Keys[ind], UserKeys_ProgressPercent[ind]] for ind in 
                    range(0, len(UserKeys_ProgressPercent))]
    
    #UserKeys_New = np.concatenate((UserKeys_Times, UserKeys_Keys, UserKeys_ProgressPercent), axis = 0)
    
    
    return UserKeys_New
        

In [4]:
def ComputeDwellTime(userKeys):
    # modify userKeys to include a column of time instead of progress pct, which is dependent on the then dwell time
    
    TimeDwellOrig = 800
    
    timeDwell = TimeDwellOrig
    nKey = -1
    for key in userKeys:
        nKey = nKey + 1
        #print(key[1])
        if key[1] == 'IncreaseDwellTime':
            if float(key[2]) == 1:
                timeDwell = timeDwell + 100
        elif key[1] == 'DecreaseDwellTime':
            #print(key[2])
            if float(key[2]) == 1:
                timeDwell = timeDwell - 100
        else:
            userKeys[nKey].append(str(float(key[2])*timeDwell))
    
    return userKeys

In [5]:
def timeTypingStart(userKeys):
    # From the user keys, find when the user actually starts typing, after having looked at the phrase and all the other 
    # function keys
    
    
    timeTypingStartInd = 0
    
    timeTypingStartIndList = list()
            
    timeUserTimeInd = 0
    
    ind = 0
    # Get start time of first trial
    
    while ind < len(userKeys):
        #print(len(userKeys[ind][1]))
        if len(userKeys[ind][1]) > 1:
            ind = ind + 1
        else:
            timeTypingStartInd = ind
            timeTypingStartIndList.append(ind)
            break
    
    #print(timeTypingStartInd)
    # Get every next phrase start timings
    while ind < len(userKeys):
        
        if userKeys[ind][1] == 'NextPhrase' and float(userKeys[ind][2]) == 1:
            
            #timeTypingStartIndList.append(ind+1)
            for ind2 in range(ind+1, len(userKeys)):
                if len(userKeys[ind2][1]) > 1:
                    ind = ind + 1
                    continue
                elif userKeys[ind2][1] == 'NextPhrase' and float(userKeys[ind][2]) == 1:
                    ind = ind + 1
                    continue
                else:
                    ind = ind2
                    timeTypingStartIndList.append(ind)
                    break
                    
        else:
            ind = ind + 1
            
    #print(timeTypingStartIndList)
    
    return timeTypingStartIndList

In [6]:
def stimPhrasesEdit_expt2019Dec(PhraseLog, subjName):
    
    # Now extract phrases from the phrase file
    phraseStim_Phrases = [item[1] for item in PhraseLog]
    phraseStim_PhrasesReduced = sorted(set(phraseStim_Phrases), key=phraseStim_Phrases.index)
    
    PhraseLogReduced= list()
    ind = -1
    
    #print(phraseStim_PhrasesReduced)
    
    for i in phraseStim_PhrasesReduced:
        ind = ind + 1

        if ind == len(phraseStim_PhrasesReduced)-1:
            if subjName == 'sa\\Test_woChinRest\\p1':
                PhraseLogReduced.append([phraseLog[phraseStim_Phrases.index(i)][0], i[0:31]])
                
            elif subjName == 'rh\\Test_wChinRest\\p1':
                PhraseLogReduced.append([phraseLog[phraseStim_Phrases.index(i)][0], i[0:-9]])
            else:
                PhraseLogReduced.append([phraseLog[phraseStim_Phrases.index(i)][0], i])
        else:
            PhraseLogReduced.append([phraseLog[phraseStim_Phrases.index(i)][0], i])
            
    del PhraseLogReduced[0]
    del PhraseLogReduced[0]
    

    
    if PhraseLogReduced[-1][1] == 'THE EXPERIMENT IS NOW DONE':
        del PhraseLogReduced[-1]
        
    if subjName == 'rh\\Test_wChinRest\\p2':
        #print(PhraseLogReduced[2])
        del PhraseLogReduced[2]
        #print(PhraseLogReduced[-1])
        del PhraseLogReduced[-1]
        
    #if subjName == 'sa\\Test_woChinRest\\p1' or subjName == 'rh\\Test_wChinRest\\p1':
    #    del PhraseLogReduced[-1]
        
    if subjName == 'sa\\Test_woChinRest\\p2':
        del PhraseLogReduced[3]
        del PhraseLogReduced[1]
        del PhraseLogReduced[0]
        
    return PhraseLogReduced
    
    

In [7]:
def scratchPadPhraseEdit_expt2019Dec(phraseUser, subjName):
    phraseUserEnd = list()
    
    print(subjName)
    
    #print(phraseUser)
    for row_ind in range(0, len(phraseUser)):
        if row_ind!= 0 and phraseUser[row_ind][1] == '':
            if len(phraseUser[row_ind-1][1])>10:
                phraseUserEnd.append(phraseUser[row_ind-1])
                #print(phraseUser[row_ind-1])
    
    if subjName == 'sa\\Test_woChinRest\\p1':
        phraseUserEnd.append([phraseUser[-1][0], phraseUser[-1][1][0:35]])
    
    if subjName == 'rh\\Test_wChinRest\\p1':
        phraseUserEnd.append(phraseUser[-1])
        
    # remove first trial of text composition
    del phraseUserEnd[0]
        
    if subjName == 'rh\\Test_wChinRest\\p2':
        print('deleting: ', phraseUserEnd[-2])
        del phraseUserEnd[-2]
        
    return phraseUserEnd

In [8]:
# This function will return the datetime in items which is the closest to the date pivot
def nearestTimePoint(dates, date):
    
    for d in dates:
        if d < date:
            nearestTP = d
        else:
            continue
    try: 
        nearestTP
        nearestTPind = dates.index(nearestTP)
    except:
        nearestTP = 0
        nearestTPind = -1
        
    return nearestTP, nearestTPind

In [9]:
def rindex(mylist, myvalue):
    return len(mylist) - mylist[::-1].index(myvalue) - 1

In [69]:
def NumberOfReadTextEventsPerKey(UserKeys, PhrasesStim, SubjectName):
    # counts the number of keys selected, because the progress reached 1
    # counts the number of keys started to select, because the progress is <0.02 (for dt=800)
    # subtracts them to find the keys_not_selected
    # also find the number of times ScratchPad was looked at, during the trial 
    # also finds the number of times a suggestion was searched for, to complete the word
    
    UserKeysTime = [val[0] for val in UserKeys]
    
    # dwell time changed or not?
    DwellTimeKeysSelected_tuple = [(ind, key[1][0:8]) for ind, key in enumerate(UserKeys) if key[2] == 1 and 'DwellTime' in key[1]]
    
    # create current dwell-time list
    ind_dtStart = 0
    dt_start = 800
    dt_list = list()
    for tuple_key, tuple_value in DwellTimeKeysSelected_tuple:
        dt_list.extend([dt_start]*(tuple_key-ind_dtStart))
        if tuple_value == 'Decrease':
            dt_start = dt_start - 100
        else:
            dt_start = dt_start + 100
        ind_dtStart = tuple_key
    
    if len(dt_list) < 1:
        dt_list = [dt_start]*len(UserKeys)
    
    elif len(dt_list) < len(UserKeys):
        dt_list.extend([dt_start]*(len(UserKeys)-len(dt_list)))
            
    nLettersNotSelected = list()
    
    
    dict_DT_progressPct = {'800' : '0.02',
                           '700' : '0.03',
                           '600' : '0.029',
                           '500' : '0.034',
                           '400' : '0.039',
                           '300' : '0.049',
                           '200' : '0.06'
                          } 
    
    
    
    
    nTimes_ScratchPad = list()
    
    # find starting point of every phrase 
    for ind_phrase in range(0,len(PhrasesStim)):
        #print(' ')
        #print('phrase : ', PhrasesStim[ind_phrase][1])
        timePhraseStarts = PhrasesStim[ind_phrase][0]
        
        # Find the time when the phrase starts
        timePhraseStartsWUserKeys, ind_start = nearestTimePoint(UserKeysTime, timePhraseStarts)
        
        
        # find when the phrase ends:
        if ind_phrase == len(PhrasesStim)-1: # if it is the last sentence, reverse index searching starts from the last index
            ind_end = len(UserKeys) - 1
            if 'sa\\Test_woChinRest\\p1' in SubjectName:
                ind_end_time, ind_end = nearestTimePoint(UserKeysTime, '2018-12-04T13:47:56.5055472+01:00')
        else: # if it's not the last sentence, reverse index searching starts from the next phrase
            ind_end_time, ind_end = nearestTimePoint(UserKeysTime, PhrasesStim[ind_phrase+1][0])
         
        
        
        keysSelected_all = [UserKeys[ind] for ind in range(ind_start, ind_end) if UserKeys[ind][2] == 1]
        #print(len(keysSelected_all))
        keysSelected = [key for key in keysSelected_all if len(key[1]) == 1 or key[1] == 'BackOne' or key[1] == 'BackMany' or key[1] == 'SpaceBar' or key[1] == 'ScratchPad' or 'Suggestion' in key[1]]
        
        # remove ScratchPad from the keysSelected, if present (if looked at the ScratchPad for so long)
        keysSelected_woScratchPad = [key for key in keysSelected if key[1] != 'ScratchPad']
        
        
        # keys for which activation and fixation was started
        keysStarted_all = [UserKeys[ind] for ind in range(ind_start, ind_end) if float(UserKeys[ind][2]) < float(dict_DT_progressPct[str(dt_list[ind_start+10])])]
        keysStarted_key = [key[1] for key in keysStarted_all]
        
        nTimes_ScratchPad_trial = keysStarted_key.count('ScratchPad')
        
        nTimes_ScratchPad.append(nTimes_ScratchPad_trial)
        
        
        
    
        
        
    return nTimes_ScratchPad

In [71]:
metricComputed = 'readTextEventPerCharacter'
#dataFolderName = r'D:\Data'
dataFolderName = r'C:\DTU\Data\201812_ExptToCheckMovementEffect\Data'

j = 0
TimeDwellAvg = 0
#nSub = 0
TypingSpeed = list()
Names = list()
read_text_event_per_character_list_wChinRest = list()
read_text_event_per_character_list_woChinRest = list()

for root, dirs, subfolder in os.walk(dataFolderName):
    
    technique = 'dwell_time'
    
    if not dirs:
        
        #if 'notCompleted' in root or 'notInclude' in root: # Some subjects do not have gaze log and have been marked as 
        
        if 'tb' in root or 'trial' in root or 'he\\' in root:
            continue
            
        #if 'ph' not in root:
        #    continue
            
        scratchPad = None
        userKeys = None
        phraseLog = None
        
        for file in subfolder:
            
            if fnmatch.fnmatch(file, 'ScratchPadLog*'):
                try:
                    fScratchPad = open(root + '\\' + file, encoding='utf-8', newline='')
                    readerScratchPad = csv.reader(fScratchPad, quotechar=None)
                    scratchPad = list(readerScratchPad)
                except:
                    if fScratchPad is not None:
                        fScratchPad.close()
                    else:
                        print('error in opening the scratchpad log file')
            
            
            if fnmatch.fnmatch(file, 'user*'):
                try:
                    fUserKey = open(root + '\\' + file, encoding='utf-8',  newline='')
                    readerUserKey = csv.reader(fUserKey, quotechar=None)
                    userKeys = list(readerUserKey)
                    userKeys.remove(userKeys[0])
                except:
                    if fUserKey is not None:
                        fUserKey.close()
                    else:
                        print('error in opening the user key log file')
                        
            if fnmatch.fnmatch(file, 'phrase*'):
                try:
                    fPhraseLog = open(root + '\\' + file, encoding='utf-8')
                    readerPhraseLog = csv.reader(fPhraseLog, quotechar=None)
                    phraseLog = list(readerPhraseLog)
                    
                except:
                    if fPhraseLog is not None:
                        fPhraseLog.close()
                    else:
                        print('error in opening the phrase log file')
                        
            if fnmatch.fnmatch(file, 'ReplacementPhraseLog.csv'):
                
                try:
                    fPhraseLog = open(root + '\\' + file)
                    readerPhraseLog = csv.reader(fPhraseLog)
                    phraseLog = list(readerPhraseLog)
                        
                except:
                    if fPhraseLog is not None:
                        print('closing')
                        fPhraseLog.close()
                    else:
                        print('error in opening the replacement phrase log file')
                        
            if fnmatch.fnmatch(file, 'multiKey*'):
                technique = 'multiKey_selection'
                    
                     
        if scratchPad is None or userKeys is None or phraseLog is None:
            continue
        else:
            #a = re.compile('(?<=201901_JanuaryExpt\\\\)(.*)(?=\\\\[1-9])')
            a = re.compile('(?<=ExptToCheckMovementEffect\\\\Data\\\\)(.*)(?=\\\\2018-1)')
            subjName = a.findall(root)[0]
            print('subject id: ', subjName)
            print(root)
            
            # a participant had problem with the computer, so the last phrase is not documented completely
            if 'sa\\Test_woChinRest\\p1' in subjName:
                scratchPad = scratchPad[0:-1]
            
            # fix scratchpad due to comma related file changes
            scratchPad_new = FixScratchPad(scratchPad)
            
            # fix phraselog due to comma related file changes
            phraseLog_new = FixScratchPad(phraseLog)
            
            
            # fix userKeys due to comma related file changes
            userKeys_new = FixUserKeys(userKeys)
                
            # find dwell time of typing
            userKeys_wDwellTime = ComputeDwellTime(userKeys_new)
                
            phraseStim_reduced = stimPhrasesEdit_expt2019Dec(phraseLog_new, subjName)
            
            
            phraseUserEnd_reduced = scratchPadPhraseEdit_expt2019Dec(scratchPad_new, subjName)
                
                                
            # check if the number of sentences match in the input and the output
            #if len(phraseUserEnd_reduced)!=len(phraseStim_reduced):
            #    phraseStim_toMatch, phraseUserEnd_toMatch = MatchPhraseNumbers(phraseStim_reduced, phraseUserEnd_reduced)
            
            # Find the number of correct letters typed
            #ComputeCorrectLetters(phraseStim_reduced, phraseUserEnd_reduced)
            
            
            # Find letters not selected:
            # Find all keys that are not selected, just shortly looked at 
            read_text_event = NumberOfReadTextEventsPerKey(userKeys_wDwellTime, phraseStim_reduced, subjName)
            # LettersNotSelected(userKeys_wDwellTime, phraseStim_reduced)
            
            
            
            if 'sa\\Test_woChinRest' in subjName or 'rh\\Test_wChinRest' in subjName:
                    if 'p1' in root:
                        
                        if 'rh' in subjName:
                            read_text_event1 = list()
                            phraseUserEnd_reduced1 = list()
                            read_text_event1 = read_text_event
                            phraseUserEnd_reduced1 = phraseUserEnd_reduced
                            
                            #print(len(phraseUserEnd_reduced1))
                        else:
                            read_text_event1 = list()
                            read_text_event1 = read_text_event
                            #print('len', len(letters_not_selected1))
                            phraseUserEnd_reduced1 = phraseUserEnd_reduced
                           
                        
                    else:
                        read_text_event2 = read_text_event
                        phraseUserEnd_reduced2 = phraseUserEnd_reduced
                   
            if subjName == 'sa\\Test_woChinRest\\p2' or subjName == 'rh\\Test_wChinRest\\p2':
                #print('Addition of lists')
                #print(phraseUserEnd_reduced1)
                read_text_event = read_text_event1 + read_text_event2 
                #print(len(phraseUserEnd_reduced1), len(phraseUserEnd_reduced2))
                phraseUserEnd_reduced = phraseUserEnd_reduced1 + phraseUserEnd_reduced2 
                #print(len(letters_not_selected1), len(letters_not_selected2), len(letters_not_selected)) 
               
            if 'p1' in subjName:
                continue
            
            #print(len(read_text_event), len(phraseUserEnd_reduced))
            print(phraseUserEnd_reduced)
            #print(read_text_event)
            
            read_text_event_per_character = [read_text_event[ind]/len(phraseUserEnd_reduced[ind][1]) for ind in range(0, len(phraseUserEnd_reduced))]
            
            print(read_text_event_per_character)    
                
            print('')
            
            if 'wChinRest' in subjName:
                print('add to list', subjName)
                read_text_event_per_character_list_wChinRest.append(read_text_event_per_character)
            else:
                read_text_event_per_character_list_woChinRest.append(read_text_event_per_character)
                
                

subject id:  bh\Test_wChinRest
C:\DTU\Data\201812_ExptToCheckMovementEffect\Data\bh\Test_wChinRest\2018-12-6-14-7-18
bh\Test_wChinRest
[['2018-12-06T14:16:44.8626944+01:00', 'Hvis sådan  et magtskifte sker,  kan Mikael Laudrup blive nød til at aflyse lansdkamps rejsen.'], ['2018-12-06T14:21:18.0607040+01:00', 'På universitetet  kaldesben klasse et hold,  et klasselokale et auditorium eller undervisningsblokale og læreren kaldes underviser .'], ['2018-12-06T14:22:45.8904197+01:00', 'Peter har været mig et dyrt bekændskab.'], ['2018-12-06T14:25:41.4683100+01:00', "I tideligere tider titulerede eleverne læreren 'hr.proffesor'."], ['2018-12-06T14:27:58.7716829+01:00', 'Hermed modtager I arbejdsgruppens invitation til Debatdag.'], ['2018-12-06T14:32:06.2704125+01:00', "Hun måtte tage tilløb i flere år men da hun hørte om master i sundheds tæknologi tænkte hun 'yes'"], ['2018-12-06T14:35:00.3448469+01:00', 'Stadig flere afgørelser skal forbi dena fuldmægtiges  bord.'], ['2018-12-06T14:37:36.

subject id:  rh\Test_wChinRest\p2
C:\DTU\Data\201812_ExptToCheckMovementEffect\Data\rh\Test_wChinRest\p2\2018-11-30-11-23-37
rh\Test_wChinRest\p2
deleting:  ['2018-11-30T11:38:35.8784680+01:00', 'Efterårssemet']
[['2018-11-29T13:02:56.0130905+01:00', 'Hermed modtager I arbejdsgruppens forslag til indbydelsen til debatdagen.'], ['2018-11-29T13:07:19.4045220+01:00', 'Det er næppe to år siden,  at Arendtsen kom til mig i min ensomme hytte,  hvor jeg ikke havde andet selskab end min kummer og '], ['2018-11-30T11:26:31.5341884+01:00', 'Det er mere traditionelt og hyggeligt med masser af landbrug i det sydlige Frankrig .'], ['2018-11-30T11:28:06.9964610+01:00', 'Christian indtog sin mad i kantinen.'], ['2018-11-30T11:30:08.2986079+01:00', 'På denne lørdag aften malede de studerende byen rød.'], ['2018-11-30T11:31:04.8278619+01:00', 'Vi har cyklet længe.'], ['2018-11-30T11:35:17.7075703+01:00', 'Ved denne form for behandling,  burde man skulle forvente en vis vekselvirkning mellem zink og mag

In [76]:
# save data to file

participant_id = [1, 2, 3, 4, 5]
n_trials = 9

participant_id_list = [[1]*n_trials, [2]*n_trials, [3]*n_trials, [4]*n_trials, [5]*n_trials]

# flatten lists

wChinRest = [1]*n_trials*len(participant_id)
ReadTextEventsPerCharacter_wChinRest = [item for sublist in read_text_event_per_character_list_wChinRest for item in sublist]

woChinRest = [1]*n_trials*len(participant_id)
ReadTextEventsPerCharacter_woChinRest = [item for sublist in read_text_event_per_character_list_woChinRest for item in sublist]

# join lists of with and without chin rest
ReadTextEventsPerCharacter_total = ReadTextEventsPerCharacter_wChinRest + ReadTextEventsPerCharacter_woChinRest

# create a pandas dataframe
df_data = pd.DataFrame({'ReadTextEventsPerChar': ReadTextEventsPerCharacter_total})

# append to file
# first find the number of rows in the existing file
filePath_PythonData = "C:\\DTU\\Data\\201812_ExptToCheckMovementEffect\\Data\\python_data2.xlsx"
loc = (filePath_PythonData) 
  
sheet = xlrd.open_workbook(loc).sheet_by_index(0)

# Extracting number of rows 
print(sheet.ncols) 

sheet_columns = [sheet.row(0)[i].value for i in range(0, sheet.ncols)]

df_existing = pd.read_excel(filePath_PythonData)
    
if metricComputed not in sheet_columns:
    filePath_PythonData_new = filePath_PythonData
    print('new metric added')

else:
    filePath_PythonData_new = filePath_PythonData[0:-5] + '_new' + filePath_PythonData[-5:]
    print('This metric is already calculated and saved: ', metricComputed)
    
df_existing[metricComputed] = ReadTextEventsPerCharacter_total
df_existing.to_excel(filePath_PythonData_new ,index=False)

13
new metric added


In [21]:
print(ans_list)

[0.6610169491525424, 1.0, 1.1204819277108433, 0.48148148148148145, 1.273972602739726, 1.2149532710280373, 1.671641791044776, 0.6470588235294118, 0.7894736842105263, 0.41935483870967744, 1.9174311926605505, 3.4150943396226414, 2.5833333333333335, 1.8085106382978724, 1.7818181818181817, 1.421487603305785, 2.3333333333333335]


In [ ]:
# check using character levenshtein distance
                dissimilarity_check_2ndLastStimPhrase = distance.levenshtein(phraseLog_reduced[-2][1].lower(), phraseUserEnd_reduced[-1][1].lower())
                dissimilarity_check_LastStimPhrase = distance.levenshtein(phraseLog_reduced[-1][1].lower(), phraseUserEnd_reduced[-1][1].lower())
                
                print(dissimilarity_check_LastStimPhrase, dissimilarity_check_2ndLastStimPhrase)
                if dissimilarity_check_LastStimPhrase > dissimilarity_check_2ndLastStimPhrase:
                    del phraseLog_reduced[-1]

In [ ]:
The large scale orchestral introduction made use of the well known technique
Instead the organisation may blame its failures on the person it failed,  even unfairly damaging the person's reputation

In [ ]:
def phrasesTyped(your_list, timeList, userKeys, stimPhraseList):
    # Find entries in scratchpad (end of each trial)
    
    IndexNotInclude = list()
    PhraseList = list()
    TypingSpeed = list()

    # After every complete phrase, the next phrase is empyty ''
    for row_ind in range(0, len(your_list)):
        if your_list[row_ind][1] == '':
            if row_ind == 1:# first empty phrase needs to be not considered , at least for most of the subjects
                continue
            if your_list[row_ind][1] == your_list[row_ind-1][1]:
                continue
            IndexNotInclude.append(row_ind)
    
    # Find the time taken for each phrase  

    #print(len(timeList), len(stimPhraseList), len(IndexNotInclude))
    #print(stimPhraseList)
    if len(IndexNotInclude) != len(timeList) :
        #print(IndexNotInclude, timeList)
        print('error and both are not same')
        #print(IndexNotInclude)
        #print(scratchPad)
    else:
        
        for indPhrase in range(0, len(IndexNotInclude)):
            startPhrase = timeList[indPhrase]    
            endPhrase = IndexNotInclude[indPhrase]-1
            headStartPhrase,sep,tail = userKeys[startPhrase][0].partition('+')
            headEndPhrase,sep,tail = your_list[endPhrase][0].partition('+')
            timeTaken = datetime.datetime.strptime(re.sub('[:.T]','-',headEndPhrase[:-1]), "%Y-%m-%d-%H-%M-%S-%f") - datetime.datetime.strptime(re.sub('[:.T]', '-', headStartPhrase[:-1]), "%Y-%m-%d-%H-%M-%S-%f")
            phraseTyped = your_list[endPhrase][1]
            PhraseList.append([timeTaken, phraseTyped])
            #print(startPhrase, ',', endPhrase, ',', headStartPhrase, ',', headEndPhrase, ',', timeTaken, ',', phraseTyped)
            
        
    #print(timeTaken)
#    print(PhraseList)
    
    # Compute the total time for the phrase and the total number of words typed during that time (1 word = 5 characters, including space)
    timeAdd = datetime.datetime.strptime('1000-1-1-0-00-00-000000',"%Y-%m-%d-%H-%M-%S-%f")
    characterLen = 0
    
    for indPhrase in range(0, len(IndexNotInclude)):
        timeAdd += PhraseList[indPhrase][0]
        characterLen += len(PhraseList[indPhrase][1])
#        print(timeAdd, ': ', characterLen)
#    print(timeAdd, ': ', characterLen)
        
    TimeTaken = (timeAdd.microsecond/1000000 + timeAdd.second)/60 + timeAdd.minute
#    print(TimeTaken)
    WPM = (characterLen)/(5*TimeTaken)
    print("Typing speed is", WPM)
    
    return WPM

In [ ]:
def AddDataToFile(metricComputed, Names, DataEvaluated):

    # Save the data
    fileIn = r'C:\DTU\Data\201805_HealthnRehab\data_summary.csv'

    if Path(fileIn).is_file():
        print(fileIn)
        fSize = os.path.getsize(fileIn) # or pd.read_excel(filename) for xls file

        # file exists
        # check if the given column exists
        with open(fileIn, 'r', newline='') as csvfileRead:
            if fSize > 0:
                fileOut = fileIn[:-4] + '1.csv'
                with open(fileOut, 'w', newline='') as csvfileWrite:
                    for row in csv.reader(csvfileRead):
                        if metricComputed in row:
                            # file exists and the metric is present in the file
                            print('This metric is already calculated and saved: ', metricComputed)
                            break
                        else:
                            # file exists but the metric is not present
                            print('adding the metric')
                            filewriter = csv.writer(csvfileWrite, delimiter = ',')
                            if len(row) > 1:
                                rowJoined = list()
                                for rowElement in row:
                                    rowJoined.append(rowElement)
                                
                                
                                print(rowJoined[0])
                                
                                if rowJoined[1] == 'subject_name':
                                    # write the titles of the metrics
                                    rowJoined.append(metricComputed)
                                    filewriter.writerow(rowJoined)
                                else:
                                    #filewriter.writerow([row, str(DataEvaluated[Names.index(row[0])])])
                                    rowJoined.append(str(DataEvaluated[Names.index(row[1])]))
                                    filewriter.writerow(rowJoined)
                                
                            else:
                                if row[1] == 'subject_name':
                                    # write the titles of the metrics
                                    filewriter.writerow([row, metricComputed])
                                else:
                                    filewriter.writerow([row, str(DataEvaluated[Names.index(row[1])])])
            
            
            else:
                fileOut = fileIn
                print('recreating a file')

                with open(fileSave, 'w', newline='') as csvfileWrite:
                    filewriter = csv.writer(csvfileWrite, delimiter=',')
                    filewriter.writerow(['subject_name', metricComputed])
                    for ind in enumerate(Names):
                        filewriter.writerow([Names[ind], DataEvaluated[ind[0]]])
                
    else:
        print('creating a file')
        a = re.compile('(?<=TypingData)(.*)(?=OptiKey)')
        with open(fileSave, 'w', newline='') as csvfileWrite:
            filewriter = csv.writer(csvfileWrite, delimiter=',')
            filewriter.writerow(['subject_name', metricComputed])
            for ind in enumerate(Names):
                subjName = a.findall(Names[ind[0]])
                filewriter.writerow([subjName, DataEvaluated[ind[0]]])
    

In [ ]:
def phrasesTyped2(your_list):
    # Find entries in scratchpad that are the start and end of the phrase
    
    IndexNotInclude = list()
    
    # After every complete phrase, the next phrase is empyty '', and the first phrase is 'scratchPadText'
    for row_ind in range(0, len(your_list)):
        if your_list[row_ind][1] in NotInclude:
            if row_ind == 0:
                IndexNotInclude.append(row_ind) # first element is to be considered for the first phrase start time
                continue
            if your_list[row_ind][1] == your_list[row_ind-1][1]:
                continue
            IndexNotInclude.append(row_ind)
    
    for row_ind in range(0, len(your_list)):
        list_inner = your_list[row_ind]
        if list_inner[1] in NotInclude:
            IndexNotInclude.append(row_ind)
    
    # Remove consecutive entries with blank scratchpads

    IndexNotIncludeArray = np.asarray(IndexNotInclude)
    IndDiff = np.diff(IndexNotIncludeArray,1)
    
    ind = len(IndexNotInclude)-1
    while (ind>-1):
        if IndDiff[ind] == 1:
            for i_2 in range (ind,len(IndexNotInclude)):
                IndexNotInclude[i_2] = IndexNotInclude[i_2]-1
        
            del(IndexNotInclude[ind])
            del your_list[IndexNotIncludeArray[ind]]
        ind = ind - 1
        
        # Find the time taken for each phrase
    PhraseList = list()

    for indPhrase in range(0, len(IndexNotInclude)-1):
        startPhrase = IndexNotInclude[indPhrase]
        endPhrase = IndexNotInclude[indPhrase+1]
        headStartPhrase,sep,tail = your_list[startPhrase+1][0].partition('+')
        headEndPhrase,sep,tail = your_list[endPhrase-1][0].partition('+')
        print(headStartPhrase, headEndPhrase)
        timeTaken = datetime.datetime.strptime(re.sub('[:.T]','-',headEndPhrase[:-1]), "%Y-%m-%d-%H-%M-%S-%f") - datetime.datetime.strptime(re.sub('[:.T]', '-', headStartPhrase[:-1]), "%Y-%m-%d-%H-%M-%S-%f")
        phraseTyped = your_list[endPhrase-1][1]
        PhraseList.append([timeTaken, phraseTyped])
        
    #print(timeTaken)
    #print(PhraseList)
    
    # Compute the total time for the phrase and the total number of words typed during that time (1 word = 5 characters, including space)
    timeAdd = datetime.datetime.strptime('1000-1-1-0-00-00-000000',"%Y-%m-%d-%H-%M-%S-%f")
    characterLen = 0

    for indPhrase in range(0, len(IndexNotInclude)-1):
        timeAdd += PhraseList[indPhrase][0]
        characterLen += len(PhraseList[indPhrase][1])
    
    TimeTaken = (timeAdd.microsecond/1000000 + timeAdd.second)/60 + timeAdd.minute
    WPM = (characterLen)/(5*TimeTaken)
    #print("Typing speed is", WPM)